In [2]:
import pandas as pd
import os
import random
import numpy as np
import timeit
import matplotlib.pyplot as plt
from citipy import citipy
import requests
from config import weather_api_key
from datetime import datetime
import time


In [4]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs


In [5]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)
print(len(coordinates))

1500


In [6]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)
print(len(cities))

606


In [9]:
print(cities[:30])

['rikitea', 'qaanaaq', 'busselton', 'nizhneyansk', 'ushuaia', 'bluff', 'east london', 'atar', 'wolmaranstad', 'luderitz', 'albany', 'quatre cocos', 'porto-vecchio', 'meadow lake', 'puerto ayora', 'atuona', 'mataura', 'matara', 'westport', 'lagoa', 'santa', 'diamantino', 'yumen', 'cape town', 'huaidian', 'nanortalik', 'chuy', 'yellowknife', 'lebu', 'dhidhdhoo']


In [3]:
url = "https://api.openweathermap.org/data/2.5/weather?&appid=" + weather_api_key


In [10]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | rikitea
Processing Record 2 of Set 1 | qaanaaq
Processing Record 3 of Set 1 | busselton
Processing Record 4 of Set 1 | nizhneyansk
City not found. Skipping...
Processing Record 5 of Set 1 | ushuaia
Processing Record 6 of Set 1 | bluff
Processing Record 7 of Set 1 | east london
Processing Record 8 of Set 1 | atar
Processing Record 9 of Set 1 | wolmaranstad
City not found. Skipping...
Processing Record 10 of Set 1 | luderitz
Processing Record 11 of Set 1 | albany
Processing Record 12 of Set 1 | quatre cocos
Processing Record 13 of Set 1 | porto-vecchio
Processing Record 14 of Set 1 | meadow lake
Processing Record 15 of Set 1 | puerto ayora
Processing Record 16 of Set 1 | atuona
Processing Record 17 of Set 1 | mataura
Processing Record 18 of Set 1 | matara
Processing Record 19 of Set 1 | westport
Processing Record 20 of Set 1 | lagoa
Processing Record 21 of Set 1 | santa
Processing Record 22 of Set 

In [11]:
city_data_df = pd.DataFrame(city_data)

city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.1203,-134.9692,296.07,72,72,5.74,PF,2022-12-01 00:54:02
1,Qaanaaq,77.4840,-69.3632,242.72,80,15,1.55,GL,2022-12-01 00:54:02
2,Busselton,-33.6500,115.3333,292.50,68,90,4.08,AU,2022-12-01 00:54:02
3,Ushuaia,-54.8000,-68.3000,281.96,57,20,8.75,AR,2022-12-01 00:51:34
4,Bluff,-46.6000,168.3333,283.60,64,40,8.99,NZ,2022-12-01 00:54:04
5,East London,-33.0153,27.9116,294.57,90,100,1.12,ZA,2022-12-01 00:54:04
6,Atar,20.5169,-13.0499,293.89,17,6,3.05,MR,2022-12-01 00:54:05
7,Luderitz,-26.6481,15.1594,290.73,73,2,4.51,NA,2022-12-01 00:54:06
8,Albany,42.6001,-73.9662,279.59,77,79,5.98,US,2022-12-01 00:49:28
9,Quatre Cocos,-20.2078,57.7625,295.42,73,76,2.63,MU,2022-12-01 00:54:06


In [16]:
new_order = ['City', 'Country', 'Date', 'Lat', 'Lng', 'Max Temp', \
    'Humidity', 'Cloudiness', 'Wind Speed']

city_data_df = city_data_df[new_order]

city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Rikitea,PF,2022-12-01 00:54:02,-23.1203,-134.9692,296.07,72,72,5.74
1,Qaanaaq,GL,2022-12-01 00:54:02,77.4840,-69.3632,242.72,80,15,1.55
2,Busselton,AU,2022-12-01 00:54:02,-33.6500,115.3333,292.50,68,90,4.08
3,Ushuaia,AR,2022-12-01 00:51:34,-54.8000,-68.3000,281.96,57,20,8.75
4,Bluff,NZ,2022-12-01 00:54:04,-46.6000,168.3333,283.60,64,40,8.99
5,East London,ZA,2022-12-01 00:54:04,-33.0153,27.9116,294.57,90,100,1.12
6,Atar,MR,2022-12-01 00:54:05,20.5169,-13.0499,293.89,17,6,3.05
7,Luderitz,NA,2022-12-01 00:54:06,-26.6481,15.1594,290.73,73,2,4.51
8,Albany,US,2022-12-01 00:49:28,42.6001,-73.9662,279.59,77,79,5.98
9,Quatre Cocos,MU,2022-12-01 00:54:06,-20.2078,57.7625,295.42,73,76,2.63


In [18]:
# Create the output file (CSV).
output_data_file = "cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")